In [ ]:
# ============================================
# 1. 필요한 AutoGen 라이브러리 임포트
# ============================================

# RoundRobinGroupChat: 여러 에이전트가 순서대로 대화하는 그룹 채팅 시스템
# - 에이전트들이 라운드 로빈 방식(순환)으로 차례대로 응답합니다
from autogen_agentchat.teams import RoundRobinGroupChat

# AssistantAgent: 특정 역할을 수행하는 AI 에이전트
# - system_message를 통해 에이전트의 역할과 행동을 정의합니다
from autogen_agentchat.agents import AssistantAgent

# OpenAIChatCompletionClient: OpenAI API를 사용하여 LLM과 통신
# - GPT 모델과의 연결을 담당합니다
from autogen_ext.models.openai import OpenAIChatCompletionClient

# 종료 조건 클래스들
# - MaxMessageTermination: 최대 메시지 수에 도달하면 대화 종료
# - TextMentionTermination: 특정 텍스트(예: "TERMINATE")가 나오면 대화 종료
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination

# Console: 대화 내용을 콘솔에 스트리밍으로 출력하는 UI
from autogen_agentchat.ui import Console

In [ ]:
# ============================================
# 2. LLM 모델 설정 및 에이전트 정의
# ============================================

# OpenAI GPT-4o-mini 모델 클라이언트 생성
# - 모든 에이전트가 이 모델을 사용하여 응답을 생성합니다
model = OpenAIChatCompletionClient(model="gpt-4o-mini")

# ----- 에이전트 1: ClarityAgent (명확성 전문가) -----
# 역할: 이메일의 명확성과 간결함을 개선
# - 모호한 표현 제거
# - 중복 내용 삭제
# - 문장을 명확하고 이해하기 쉽게 만들기
clarity_agent = AssistantAgent(
    "ClarityAgent",  # 에이전트 이름
    model_client=model,  # 사용할 LLM 모델
    system_message="""You are an expert editor focused on clarity and simplicity. 
            Your job is to eliminate ambiguity, redundancy, and make every sentence crisp and clear. 
            Don't worry about persuasion or tone — just make the message easy to read and understand.""",
)

# ----- 에이전트 2: ToneAgent (톤 전문가) -----
# 역할: 이메일의 감정적 톤과 전문성 조정
# - 따뜻하고 자신감 있는 표현으로 변경
# - 전문적이면서도 인간적인 느낌 유지
# - 딱딱하거나 너무 캐주얼한 표현 조정
tone_agent = AssistantAgent(
    "ToneAgent",
    model_client=model,
    system_message="""You are a communication coach focused on emotional tone and professionalism. 
            Your job is to make the email sound warm, confident, and human — while staying professional 
            and appropriate for the audience. Improve the emotional resonance, polish the phrasing, 
            and adjust any words that may come off as stiff, cold, or overly casual.""",
)

# ----- 에이전트 3: PersuasionAgent (설득 전문가) -----
# 역할: 이메일의 설득력 향상
# - 행동 심리학 기반의 설득 기법 적용
# - 명확한 Call-to-Action (CTA) 추가
# - 혜택 강조 및 약한 표현 제거
persuasion_agent = AssistantAgent(
    "PersuasionAgent",
    model_client=model,
    system_message="""You are a persuasion expert trained in marketing, behavioral psychology, 
            and copywriting. Your job is to enhance the email's persuasive power: improve call to action, structure arguments, and emphasize benefits. Remove weak or passive language.""",
)

# ----- 에이전트 4: SynthesizerAgent (통합 전문가) -----
# 역할: 이전 에이전트들의 제안을 하나로 통합
# - 명확성, 톤, 설득력의 개선 사항을 모두 반영
# - 일관성 있고 자연스러운 최종 초안 작성
# - 전문적이고 효과적인 이메일로 완성
synthesizer_agent = AssistantAgent(
    "SynthesizerAgent",
    model_client=model,
    system_message="""You are an advanced email-writing specialist. Your role is to read all 
            prior agent responses and revisions, and then **synthesize the best ideas** into a unified, 
            polished draft of the email. Focus on: Integrating clarity, tone, and persuasion improvements; 
            Ensuring coherence, fluency, and a natural voice; Creating a version that feels professional, 
            effective, and readable.""",
)

# ----- 에이전트 5: CriticAgent (품질 평가 전문가) -----
# 역할: 최종 이메일 품질 검증
# - 명확성, 흐름, 전문적 톤, CTA 효과성 평가
# - 문제가 있으면 구체적인 개선 제안 제공
# - 전문적 기준을 충족하면 "TERMINATE"로 대화 종료
critic_agent = AssistantAgent(
    "CriticAgent",
    model_client=model,
    system_message="""You are an email quality evaluator. Your job is to perform a final review 
            of the synthesized email and determine if it meets professional standards. Review the email for: 
            Clarity and flow, appropriate professional tone, effective call-to-action, and overall coherence.
            Be constructive but decisive. If the email has major flaws (unclear message, unprofessional tone, 
            or missing key elements), provide ONE specific improvement suggestion. If the email meets professional standards and communicates effectively, respond with 'The email meets professional standards.' followed by `TERMINATE` on a new line. You should only approve emails that are perfect enough for professional use, dont settle.""",
)

In [ ]:
# ============================================
# 3. 대화 종료 조건 설정
# ============================================

# 종료 조건 1: 텍스트 기반 종료
# - CriticAgent가 "TERMINATE" 텍스트를 출력하면 대화가 종료됩니다
# - 이메일이 전문적 기준을 충족했을 때 사용됩니다
text_termination = TextMentionTermination("TERMINATE")

# 종료 조건 2: 최대 메시지 수 제한
# - 최대 30개의 메시지가 교환되면 대화가 자동으로 종료됩니다
# - 무한 루프를 방지하기 위한 안전 장치입니다
max_messages_termination = MaxMessageTermination(max_messages=30)

# 종료 조건 결합 (OR 연산)
# - 두 조건 중 하나라도 만족하면 대화가 종료됩니다
# - "TERMINATE"가 나오거나 OR 30개 메시지에 도달하면 종료
termination_condition = text_termination | max_messages_termination

In [ ]:
# ============================================
# 4. 에이전트 팀 구성 및 실행
# ============================================

# RoundRobinGroupChat으로 에이전트 팀 생성
# - participants: 참여할 에이전트 목록 (순서대로 실행됨)
#   1) ClarityAgent → 2) ToneAgent → 3) PersuasionAgent 
#   → 4) SynthesizerAgent → 5) CriticAgent → (다시 1번부터 반복)
# - termination_condition: 대화 종료 조건
team = RoundRobinGroupChat(
    participants=[
        clarity_agent,      # 1단계: 명확성 개선
        tone_agent,         # 2단계: 톤 조정
        persuasion_agent,   # 3단계: 설득력 향상
        synthesizer_agent,  # 4단계: 통합 및 초안 작성
        critic_agent,       # 5단계: 품질 평가 및 승인/거부
    ],
    termination_condition=termination_condition,
)

# 팀 실행 및 결과 출력
# - task: 최적화할 이메일 초안 입력
# - run_stream(): 대화를 스트리밍 방식으로 실행
# - Console(): 실시간으로 각 에이전트의 응답을 콘솔에 출력
# - await: 비동기 실행 (Jupyter 노트북에서 필요)
await Console(
    team.run_stream(
        task="Hi! Im hungry, buy me lunch and invest in my business. Thanks."
        # 👆 여기에 개선하고 싶은 이메일 내용을 입력하세요
    )
)